In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.python.client import device_lib
import matplotlib.pyplot as plt
import h5py
import time
import numpy as np
import scipy.io as sio
from scipy import io
import mat73
import math
 
print(device_lib.list_local_devices())

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
    # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    
        print(e)

## Test

In [ ]:
N_node = 4
Bound= 1
N=math.comb(N_node,2)
N_data_test = 1000
 
num = N_node*2-3
pred_position = np.zeros((N_data_test,N_node*2))
ae_distance = np.zeros((1, N))
mse_distance = np.zeros((1, N_node))
 
start = time.time()
    
    
model = keras.models.load_model('./model.h5')
test_data = sio.loadmat('./testset.mat')
X_test = np.transpose(np.array(test_data['distance']))
target_position = np.transpose(np.array(test_data['node_position']))

pred = np.squeeze(np.transpose(np.array(model.predict(X_test))))

err_distance = np.zeros((N_data_test, N_node))

for idx in range(0, N_data_test):
    cnt=0 
    for idx2 in range(0, 2*N_node, 2):
        err_distance[idx, cnt] = np.square(pred_position[idx, idx2]-target_position[idx, idx2]) + np.square((pred_position[idx, idx2+1]-target_position[idx, idx2+1]))
        cnt=cnt+1
        
        
for idx in range(0, N_node):
    ae_distance = np.mean(np.abs(np.sqrt(err_distance[:, idx])))

mae= np.mean(ae_distance)
    
sum=0
for idx in range(0, N_node):
    mse_distance[1, idx] = np.mean(err_distance[:, idx])
    sum = sum + mse_distance[1, idx]  
    
rmse = np.sqrt(sum/N_node)    
    
print('소요 시간 :', time.time() - start)

np.savez('./performance.npz', mae=mae,mean_rmse=rmse)